# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [1]:
!pip install OpenAI
!pip install python_dotenv

from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

from google.colab import userdata
OPENAI_API_KEY  = userdata.get('OPENAI_API_KEY3')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 831.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.6 MB/s eta 0:00:00


# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [2]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [3]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

The Formula 1 World Championship in 2010 was won by Sebastian Vettel, driving for Red Bull Racing.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [4]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [8]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

#funny to see that it doesn't always follow the format provided.

The 2006 F1 Championship was won by Fernando Alonso, driving for the Renault team.


In [9]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

The 2019 F1 championship was won by Lewis Hamilton from Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [10]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [ ]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes
Points: 413


In [ ]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [ ]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


#European cup

In [15]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in soccor competition and european country tournaments.\n\n'},
    {'role':'user', 'content':'Who won the 2016 championship?'},
    {'role':'assistant', 'content':"""Country: Portugal. \nFootball star: Cristiano Ronaldo. """},
    {'role':'user', 'content':'Who won the 2012 championship?'},
    {'role':'assistant', 'content':"""Country: Spain. \nFootball star: Sergio Ramos. """},
]

print(return_OAIResponse("Who won the 2004 championship?", context_user))
print(return_OAIResponse("Who will win this year?", context_user))
print(return_OAIResponse("When dit the netherlands win?", context_user))

Country: Greece.
Key player: Angelos Charisteas.
As an expert in soccer competitions and European country tournaments, I cannot predict with certainty who will win this year's championship. The outcome of a tournament depends on various factors such as team performance, strategy, player form, and luck. It is always exciting to watch the matches unfold to see who will emerge victorious.
The Netherlands won the European Championship in 1988.


In [17]:
context_user = [
    {'role':'system', 'content':
     """You are and expert in soccor competition and european country tournaments.

     It was great to see a team as Greece to win the cup in 2004.
     Sentiment: Positive

     How can it be, that such a bad team as Greece can win the cup
     Sentiment: Negative.

     I'm not a soccer fan, so I really don't care.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("pfff again soccor... on the tele", context_user))
print(return_OAIResponse("the netherlands can play so gracious and having the best player, but will never win", context_user))

It seems like soccer is not your favorite sport. That's totally fine, everyone has their own interests!
As an expert in soccer competition and European country tournaments, I can provide some insight into the Netherlands' performance. The Netherlands has a rich soccer history and is known for producing talented players. They have had strong teams in the past, known for their stylish and attacking play. However, despite having some of the best players and playing attractive soccer, the Netherlands has faced challenges when it comes to winning major tournaments like the World Cup or the European Championship. 

There have been instances where the Netherlands has come close to winning, like in the 1974, 1978, and 2010 World Cups, where they reached the final but ultimately fell short. The team's inability to win major tournaments despite their talent and style of play has often been attributed to a combination of factors such as tough competition, lack of luck, and sometimes issues within

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [ ]:
#It appears, that the content needs to be fed exactly to produce the same structure as provided.